In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import duckdb

In [2]:
con = duckdb.connect()
con.execute("""-- Set memory limits before running query
                SET memory_limit='16GB';
                -- Enable progress tracking
                SET enable_progress_bar=true;
                -- Enable detailed profiling
                SET profiling_mode='detailed';
            """)

In [3]:
# Define file path and view name
parquet_path = "/Users/couch/intel_research/data/0000_part_00 (1).parquet"
view_name = "parquet_view"

# Create a view over the Parquet file
con.execute(f"""
    CREATE VIEW {view_name} AS 
    SELECT * FROM read_parquet('{parquet_path}');
""")

In [4]:
df = con.execute(f"SELECT * FROM {view_name} LIMIT 5").fetchdf()
print(df)

              load_ts         batch_id  \
0 2019-10-24 04:09:21  20191023-210226   
1 2019-10-24 04:09:21  20191023-210226   
2 2019-10-24 04:09:21  20191023-210226   
3 2019-10-24 04:09:21  20191023-210226   
4 2019-10-24 04:09:21  20191023-210226   

                                           audit_zip  \
0  2019102316-i-0728cf8ab79db9e58-BfZXhQdKBpG8dxo...   
1  2019102316-i-0728cf8ab79db9e58-BfZXhQdKBpG8dxo...   
2  2019102316-i-0728cf8ab79db9e58-BfZXhQdKBpG8dxo...   
3  2019102316-i-0728cf8ab79db9e58-BfZXhQdKBpG8dxo...   
4  2019102316-i-0728cf8ab79db9e58-BfZXhQdKBpG8dxo...   

                           audit_internal_path  \
0  V8_1_SYSTEMUSAGEBYFGNDAPP_20191023231918.V8   
1  V8_1_SYSTEMUSAGEBYFGNDAPP_20191023231918.V8   
2  V8_1_SYSTEMUSAGEBYFGNDAPP_20191023231918.V8   
3  V8_1_SYSTEMUSAGEBYFGNDAPP_20191023231918.V8   
4  V8_1_SYSTEMUSAGEBYFGNDAPP_20191023231918.V8   

                               guid  interval_start_utc    interval_end_utc  \
0  0028155e73c04ed2a420eb00af9

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT * FROM parquet_view LIMIT 5
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.0318s             ││
│└──────────────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0023s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││          Cte Filter Pusher: 0.0000s          ││
││             Deliminator: 0.0000s             ││


In [5]:
df.columns

Index(['load_ts', 'batch_id', 'audit_zip', 'audit_internal_path', 'guid',
       'interval_start_utc', 'interval_end_utc', 'interval_local_start',
       'interval_local_end', 'ts', 'dt', 'pid', 'proc_name', 'proc_package',
       'captioned', 'duration', 'metric_name', 'aggregation_type',
       'attribute_level1', 'nrs', 'avg_val', 'min_val', 'max_val',
       'percentile_50th', 'percentile_75th', 'percentile_90th'],
      dtype='object')

In [6]:
df = con.execute(f"SELECT DISTINCT proc_name FROM {view_name}").fetchdf()
print(df)

                              proc_name
0                    steamwebhelper.exe
1      keepuppirates-win64-shipping.exe
2                          pet idle.exe
3                             whale.exe
4                             debut.exe
...                                 ...
29177                   gamebrowser.exe
29178            marsserverprovider.exe
29179                     delftship.exe
29180             l2soft.eu updater.exe
29181          spark_2_9_4-with-jre.exe

[29182 rows x 1 columns]


┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT DISTINCT proc_name FROM parquet_view
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.121s              ││
│└──────────────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0010s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││          Cte Filter Pusher: 0.0000s          ││
││             Deliminator: 0.0000s       

In [7]:
df = con.execute(f"SELECT DISTINCT aggregation_type FROM {view_name}").fetchdf()
print(df)

   aggregation_type
0                 0
1                 2
2                 1


┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT DISTINCT aggregation_type FROM parquet_view
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.0769s             ││
│└────────────────────��─────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0017s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││          Cte Filter Pusher: 0.0000s          ││
││             Deliminator: 0.0000

In [8]:
# views allow many queries to be run on the same data without having to re-read it from disk.
create_view = """
    CREATE VIEW {view_name} AS 
        SELECT * FROM read_parquet('{parquet_path}');
"""

In [10]:
webcat_data = pq.ParquetDataset("/Users/couch/intel_research/data/webcat_pivot")
table = webcat_data.read()
print(table.schema)

guid: string
social_social_network: int64
private_private: int64
productivity_word_processing: int64
news_news: int64
social_communication: int64
productivity_spreadsheets: int64
content_creation_photo_edit_creation: int64
search_search: int64
productivity_other: int64
entertainment_music_audio_streaming: int64
finance_banking_and_accounting: int64
games_other: int64
entertainment_other: int64
education_education: int64
productivity_programming: int64
productivity_presentations: int64
reference_reference: int64
shopping_shopping: int64
other_unclassified: int64
recreation_travel: int64
entertainment_video_streaming: int64
games_video_games: int64
productivity_crm: int64
mail_mail: int64
social_communication_live: int64
content_creation_video_audio_edit_creation: int64
productivity_project_management: int64
content_creation_web_design_development: int64


In [11]:
parquet_path = "/Users/couch/intel_research/data/webcat_pivot"
view_name = "webcat_view"

con.execute(f"""
    CREATE VIEW {view_name} AS 
    SELECT * FROM read_parquet('{parquet_path}');
""")

In [12]:
con.execute(f"SELECT * FROM {view_name} LIMIT 5").fetchdf()

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT * FROM webcat_view LIMIT 5
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.0219s             ││
│└──────────────────────────���───────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0003s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││          Cte Filter Pusher: 0.0000s          ││
││             Deliminator: 0.0000s             ││

,guid,social_social_network,private_private,productivity_word_processing,news_news,social_communication,productivity_spreadsheets,content_creation_photo_edit_creation,search_search,productivity_other,...,other_unclassified,recreation_travel,entertainment_video_streaming,games_video_games,productivity_crm,mail_mail,social_communication_live,content_creation_video_audio_edit_creation,productivity_project_management,content_creation_web_design_development
0,00010ab0daf7426b813be3de1288b9a0,2466,<NA>,<NA>,26581,<NA>,<NA>,<NA>,390184,194240,...,549370,615652,452756,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,00052d76ed974971b466e32b9388c218,1921190,2967408,<NA>,14350,101456,<NA>,<NA>,300999,107500,...,1020627,<NA>,3128489,<NA>,<NA>,10731,<NA>,<NA>,<NA>,163385
2,0009c1d03194477ca9f581169f26f974,<NA>,<NA>,<NA>,46181,<NA>,<NA>,<NA>,<NA>,<NA>,...,96395,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,0010f376c3214cd2bae5ea2e3fba2d4b,2367754,<NA>,143195,67480,69127,1587912,<NA>,470707,169485,...,1152117,440325,2529469,<NA>,<NA>,819647,<NA>,<NA>,<NA>,<NA>
4,0001c8444e8a42d9a38674f737886d0e,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,261921,<NA>,...,1191621,83900,3984708,<NA>,<NA>,526094,<NA>,<NA>,<NA>,<NA>


In [14]:
parquet_path = "/Users/couch/intel_research/data/webcat_0000_part_00.parquet"
view_name = "webcat_view_V2"

con.execute(f"""
    CREATE VIEW {view_name} AS 
    SELECT * FROM read_parquet('{parquet_path}');
""")

con.execute(f"SELECT * FROM {view_name} LIMIT 5").fetchdf()

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT * FROM webcat_view_V2 LIMIT 5
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.0403s             ││
│└─────────────────────────���────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0036s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││          Cte Filter Pusher: 0.0000s          ││
││             Deliminator: 0.0000s            

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,dt,browser,parent_category,sub_category,duration_ms,page_load_count,site_count,domain_count,page_visit_count
0,2021-06-11 03:12:58,20210610-200228,2021061003-i-04a9a906e1cd5620c-Nza8Wx4VDJjXb7j...,V8_2_WEB_CAT_USAGE_20210610134711.V8,000091c0adc149389235ed2c5f15a59e,2021-06-09 03:43:09,2021-06-10 03:45:19,2021-06-09 13:43:09,2021-06-10 13:45:19,2021-06-09,chrome,reference,reference,36087,1,1,1,4
1,2021-06-11 03:12:58,20210610-200228,2021061003-i-04a9a906e1cd5620c-Nza8Wx4VDJjXb7j...,V8_2_WEB_CAT_USAGE_20210610134711.V8,000091c0adc149389235ed2c5f15a59e,2021-06-09 03:43:09,2021-06-10 03:45:19,2021-06-09 13:43:09,2021-06-10 13:45:19,2021-06-09,edge,productivity,other,21667,2,2,1,3
2,2021-06-11 03:12:58,20210610-200228,2021061003-i-04a9a906e1cd5620c-Nza8Wx4VDJjXb7j...,V8_2_WEB_CAT_USAGE_20210610134711.V8,000091c0adc149389235ed2c5f15a59e,2021-06-09 03:43:09,2021-06-10 03:45:19,2021-06-09 13:43:09,2021-06-10 13:45:19,2021-06-09,chrome,other,unclassified,296422,33,24,2,89
3,2021-06-11 03:12:58,20210610-200228,2021061003-i-04a9a906e1cd5620c-Nza8Wx4VDJjXb7j...,V8_2_WEB_CAT_USAGE_20210610134711.V8,000091c0adc149389235ed2c5f15a59e,2021-06-09 03:43:09,2021-06-10 03:45:19,2021-06-09 13:43:09,2021-06-10 13:45:19,2021-06-09,chrome,productivity,other,4924,2,1,1,2
4,2021-06-11 03:12:58,20210610-200228,2021061003-i-04a9a906e1cd5620c-Nza8Wx4VDJjXb7j...,V8_2_WEB_CAT_USAGE_20210610134711.V8,000091c0adc149389235ed2c5f15a59e,2021-06-09 03:43:09,2021-06-10 03:45:19,2021-06-09 13:43:09,2021-06-10 13:45:19,2021-06-09,chrome,search,search,80042,24,14,1,18


audit_zip         │
│    audit_internal_path    │
│            guid           │
│     interval_start_utc    │
│      interval_end_utc     │
│    interval_local_start   │
│     interval_local_end    │
│             dt            │
│          browser          │
│      parent_category      │
│        sub_category       │
│        duration_ms        │
│      page_load_count      │
│         site_count        │
│        domain_count       │
│      page_visit_count     │
│                           │
│         4096 Rows         │
│          (0.03s)          │
└───────────────────────────┘





In [16]:
con.execute(f"SELECT DISTINCT parent_category, sub_category FROM {view_name} ORDER BY parent_category").fetchdf()

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT DISTINCT parent_category, sub_category FROM webcat_view_V2 ORDER BY parent_category
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.103s              ││
│└───────���──────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0012s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││          Cte Filter Pusher: 0.0000s      

,parent_category,sub_category
0,content creation,web design / development
1,content creation,photo edit/creation
2,content creation,video/audio edit/creation
3,education,education
4,entertainment,video streaming
5,entertainment,music / audio streaming
6,entertainment,other
7,finance,banking and accounting
8,games,other
9,games,video games


In [24]:
con.execute(f"SELECT parent_category, (SUM(duration_ms)/60000) AS duration_min FROM {view_name} GROUP BY parent_category ORDER BY parent_category").fetchdf()

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT parent_category, (SUM(duration_ms)/60000) AS duration_min FROM webcat_view_V2 GROUP BY parent_category ORDER BY parent_category
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.0683s             ││
│└──────────────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0003s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
││

,parent_category,duration_min
0,content creation,3.021983e+06
1,education,1.259095e+07
2,entertainment,2.366244e+08
3,finance,5.703987e+06
4,games,1.133628e+07
5,mail,3.028810e+07
6,news,1.188503e+07
7,other,3.104597e+08
8,private,2.071968e+08
9,productivity,2.163635e+07


In [25]:
con.execute(f"SELECT parent_category, (SUM(duration_ms)/3600000) AS duration_hr FROM {view_name} GROUP BY parent_category ORDER BY parent_category").fetchdf()

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT parent_category, (SUM(duration_ms)/3600000) AS duration_hr FROM webcat_view_V2 GROUP BY parent_category ORDER BY parent_category
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.0705s             ││
│└──────────────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0003s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Materialization: 0.0000s     ││
│

,parent_category,duration_hr
0,content creation,5.036638e+04
1,education,2.098492e+05
2,entertainment,3.943740e+06
3,finance,9.506644e+04
4,games,1.889379e+05
5,mail,5.048017e+05
6,news,1.980839e+05
7,other,5.174328e+06
8,private,3.453281e+06
9,productivity,3.606059e+05


      Create Plan: 0.0000s             ││
││            Resolve Types: 0.0000s            ││
│└──────────────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│                Planner: 0.0036s                │
│┌──────────────────────────────────────────────┐│
││               Binding: 0.0036s               ││
│└──────────────────────────────────────────────┘│
└───────��────────────────────────────────────────┘
┌───────────────────────────┐
│           QUERY           │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│          ORDER_BY         │
│    ────────────────────   │
│ memory.main.webcat_view_V2│
│    .parent_category ASC   │
│                           │
│          15 Rows          │
│          (0.01s)          │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│         PROJECTION        │
│    ────────────────────   │
│      parent_category      │
│        duration_

In [26]:
con.execute(f"SELECT parent_category, sub_category, (SUM(duration_ms)/3600000) AS duration_hr FROM {view_name} GROUP BY parent_category, sub_category ORDER BY parent_category").fetchdf()

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
SELECT parent_category, sub_category, (SUM(duration_ms)/3600000) AS duration_hr FROM webcat_view_V2 GROUP BY parent_category, sub_category ORDER BY parent_category
┌────────────────────────────────────────────────┐
│┌──────────────────────────────────────────────┐│
││              Total Time: 0.130s              ││
│└──────────────────────────────────────────────┘│
└────────────────────────────────────────────────┘
┌────────────────────────────────────────────────┐
│               Optimizer: 0.0008s               │
│┌──────────────────────────────────────────────┐│
││        Build Side Probe Side: 0.0000s        ││
││           Column Lifetime: 0.0000s           ││
││           Common Aggregate: 0.0000s          ││
││        Common Subexpressions: 0.0000s        ││
││      Compressed Mater

,parent_category,sub_category,duration_hr
0,content creation,web design / development,3.728031e+04
1,content creation,video/audio edit/creation,1.516382e+03
2,content creation,photo edit/creation,1.156969e+04
3,education,education,2.098492e+05
4,entertainment,other,3.180590e+05
5,entertainment,music / audio streaming,3.271571e+04
6,entertainment,video streaming,3.592965e+06
7,finance,banking and accounting,9.506644e+04
8,games,video games,2.751150e+03
9,games,other,1.861868e+05


In [ ]:
con.execute(f"SELECT parent_category, sub_category, SUM(duration_ms) FROM {view_name} GROUP BY parent_category, sub_category ORDER BY parent_category").fetchdf()